#       Seq2Seq: Text Summarization with Keras
![](http://abigailsee.com/img/pointer-gen.png)

## Process
1. Preprocessing
2. Word2vec
3. Building Seq2Seq Architecture
4. Training with  BBC article&summary Dataset
5. Generate Summary with my_summarizer

## Step 1. Import Data

In [35]:
import numpy as np
import os
import pandas as pd
import re

In [0]:
news_category = ["business", "entertainment", "politics", "sport", "tech"]

row_doc = "/Users/akr712/Desktop/Text_Summarization_using_Keras/Row News Articles/"
summary_doc = "/Users/akr712/Desktop/Text_Summarization_using_Keras/Summaries/"

data={"articles":[], "summaries":[]}

DataFrameを作る
- テキストファイル名を取得
- テキストファイルの中身を取得
- データフレーム化

In [0]:
import os
directories = {"news": row_doc, "summary": summary_doc}
row_dict = {}
sum_dict = {}

for path in directories.values():
    if path == row_doc:
        file_dict = row_dict
    else:
        file_dict = sum_dict
    dire = path
    for cat in news_category:
        category = cat
        files = os.listdir(dire + category)
        file_dict[cat] = files

In [0]:
row_dict.keys()

dict_keys(['politics', 'sport', 'business', 'tech', 'entertainment'])

In [0]:
"001.txt" in row_dict['politics']

True

In [0]:
text = parsetext(row_doc, "sport", "291.txt")

file read successfully


In [0]:
text

'thomas out of six nations\n\nwales captain gareth thomas has been ruled out of the rest of the six nations with a broken thumb.\n\nthe full-back will have surgery on monday after fracturing his thumb in the 24-18 win over france on saturday. but welsh legend phil bennett insisted wales can cope without thomas as they chase a first grand slam in 27 years. bennett told bbc sport: "such is the spirit in the camp, they\'ll put kevin morgan at 15, rhys williams at wing and just carry on." thomas will miss the match against scotland on 13 march, and what promises to be a huge encounter against the irish six days later. bennett added: "it\'s a setback. he\'s a great captain, he leads from the front and the boys love him." thomas was replaced at half-time by williams as his side turned around a 15-6 deficit in paris.\n\n"with gareth missing i would think michael owen will be our captain," said wales coach mike ruddock. "he did a great job in the second half in france. he has been vice-captain

In [0]:
for name in row_dict["politics"]:
    print(name)

In [0]:
row_dict["sport"][0]

'289.txt'

In [0]:
len(row_dict["sport"])

511

In [0]:
row_data = {}
for cat in row_dict.keys():
    cat_dict = {}
    # row_data_frame[cat] = []
    for i in range(0, len(row_dict[cat])):
        filename = row_dict[cat][i]
        path = row_doc + cat + "/" + filename
        with open(path, "rb") as f:                
            text = f.read()
            cat_dict[filename[:3]] = text
    row_data[cat] = cat_dict

In [0]:
sum_data = {}
for cat in sum_dict.keys():
    cat_dict = {}
    # row_data_frame[cat] = []
    for i in range(0, len(sum_dict[cat])):
        filename = sum_dict[cat][i]
        path = summary_doc + cat + "/" + filename
        with open(path, "rb") as f:                
            text = f.read()
            cat_dict[filename[:3]] = text
    sum_data[cat] = cat_dict

In [0]:
type(row_data["business"])

dict

In [0]:
sum_data["business"]

In [0]:
news_business = pd.DataFrame.from_dict(row_data["business"], orient="index", columns=["row_article"])
news_business.head()

,row_article
424,b'EU-US seeking deal on air dispute\n\nThe EU ...
299,b'US to rule on Yukos refuge call\n\nYukos has...
276,b'India power shares jump on debut\n\nShares i...
272,b'Markets signal Brazilian recovery\n\nThe Bra...
379,"b""Reliance unit loses Anil Ambani\n\nAnil Amba..."


In [0]:
#  news_category = ["business", "entertainment", "politics", "sport", "tech"]
news_entertainment = pd.DataFrame.from_dict(row_data["entertainment"], orient="index", columns=["row_article"])
news_politics = pd.DataFrame.from_dict(row_data["politics"], orient="index", columns=["row_article"])
news_sport = pd.DataFrame.from_dict(row_data["sport"], orient="index", columns=["row_article"])
news_tech = pd.DataFrame.from_dict(row_data["tech"], orient="index", columns=["row_article"])

In [0]:
# summary data
summary_business = pd.DataFrame.from_dict(sum_data["business"], orient="index", columns=["summary"])
summary_entertainment = pd.DataFrame.from_dict(sum_data["entertainment"], orient="index", columns=["summary"])
summary_politics = pd.DataFrame.from_dict(sum_data["politics"], orient="index", columns=["summary"])
summary_sport = pd.DataFrame.from_dict(sum_data["sport"], orient="index", columns=["summary"])
summary_tech = pd.DataFrame.from_dict(sum_data["tech"], orient="index", columns=["summary"])

In [0]:
summary_business.head()

,summary
424,b'Both sides hope to reach a negotiated deal o...
299,b'Yukos has said a US bankruptcy court will de...
276,"b'Shares in India\'s largest power producer, N..."
272,"b""Investors have praised his handling of the e..."
379,"b""Anil, 45, has stepped down as director and v..."


In [0]:
business = news_business.join(summary_business, how='inner')
entertainment = news_entertainment.join(summary_entertainment, how='inner')
politics = news_politics.join(summary_politics, how='inner')
sport = news_sport.join(summary_sport, how='inner')
tech = news_tech.join(summary_tech, how='inner')

In [0]:
business = news_business.join(summary_business, how='inner')

In [0]:
business.head()

,row_article,summary
424,b'EU-US seeking deal on air dispute\n\nThe EU ...,b'Both sides hope to reach a negotiated deal o...
299,b'US to rule on Yukos refuge call\n\nYukos has...,b'Yukos has said a US bankruptcy court will de...
276,b'India power shares jump on debut\n\nShares i...,"b'Shares in India\'s largest power producer, N..."
272,b'Markets signal Brazilian recovery\n\nThe Bra...,"b""Investors have praised his handling of the e..."
379,"b""Reliance unit loses Anil Ambani\n\nAnil Amba...","b""Anil, 45, has stepped down as director and v..."


In [0]:
print("row", len(business.iloc[0,0]))
print("sum", len(business.iloc[0,1]))

row 2179
sum 984


In [0]:
list_df = [business, entertainment, politics, sport, tech]
length = 0
for df in list_df:
    length += len(df)

In [0]:
print("length of all data: ", length)

length of all data:  2225


In [0]:
bbc_df = pd.concat([business, entertainment, politics, sport, tech], ignore_index=True)
len(bbc_df)

2225

## Step 2. Preprocessing Text Data
1. Clean Text
2. Tokenize
3. Vocabrary
4. Padding
5. One-Hot Encoding
6. Reshape to (MAX_LEN, One-Hot Encoding DIM)

### 2-1. Clean Text

In [0]:
def cleantext(text):
    text = str(text)
    text=text.split()
    words=[]
    for t in text:
        if t.isalpha():
            words.append(t)
    text=" ".join(words)
    text=text.lower()
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

In [0]:
for col in bbc_df.columns:
    bbc_df[col] = bbc_df[col].apply(lambda x: cleantext(x))

In [0]:
bbc_df.head()

,row_article,summary
0,seeking deal on air eu and us have agreed to b...,sides hope to reach a negotiated deal over sta...
1,to rule on yukos refuge has said a us bankrupt...,has said a us bankruptcy court will decide whe...
2,power shares jump on in largest power national...,in largest power national thermal power corp h...
3,signal brazilian brazilian stock market has ri...,have praised his handling of the economy as fo...
4,unit loses anil the younger of the two brother...,has stepped down as director and of indian pet...


In [45]:
df.head()

,row_article,summary
0,continues rapid economy has expanded by a brea...,overall investment in fixed assets was still u...
1,deccan seals deccan has ordered airbus planes ...,government has given its backing to cheaper an...
2,job growth continues in us created fewer jobs ...,creation was one of last main concerns for the...
3,owner buys rival for retail giant federated de...,retail giant federated department stores is to...
4,secures giant japan is to supply japan airline...,chose the after carefully considering both it ...


In [10]:
len_list =[]
for article in articles:
    words = article.split()
    length = len(words)
    len_list.append(length)
max(len_list)

3807

### 2-2. A wrong way of tokenization: text_to_word_sequence & one_hot method
1. Tokenize: text_to_word_sequence 
2. Cut off word more than max_len 1000
3. Padding: pad_sequences
4. One-Hot Vectrizer: dimention is under vocab_size
5. Reshape: manual max_len * one-Hot matrix 

#### 2-2-1. Tokenize: text_to_word_sequence

In [0]:
from keras.preprocessing.text import text_to_word_sequence

def text2seq(text):
    seq = text_to_word_sequence(text)
    return seq

article_sequences = []
for article in articles:
    seq = text2seq(article)
    article_sequences.append(seq)

In [48]:
article_sequences[0][:10]

['continues',
 'rapid',
 'economy',
 'has',
 'expanded',
 'by',
 'a',
 'breakneck',
 'during',
 'faster']

In [0]:
sum_sequences = []
for summary in summaries:
    seq = text2seq(summary)
    sum_sequences.append(seq)

In [50]:
sum_sequences[0][:10]

['overall',
 'investment',
 'in',
 'fixed',
 'assets',
 'was',
 'still',
 'up',
 'from',
 'the']

In [13]:
sum_len = []
for i in range(1000):
    sum_len.append(len(sum_sequences[i]))
print("max", max(sum_len))
print("min", min(sum_len))
print("ave", sum(sum_len)/len(sum_len))

max 1822
min 28
ave 125.866


In [26]:
art_len = []
for i in range(1000):
    art_len.append(len(article_sequences[i]))
print("max", max(art_len))
print("min", min(art_len))
print("ave", sum(art_len)/len(art_len))

max 3693
min 75
ave 279.943


In [28]:
sum_sequences[1][:6]

['government', 'has', 'given', 'its', 'backing', 'to']

#### 2-2-2. Cut off word more than max_len 1000

In [0]:
max_1000_art_sequences = []
for sequences in article_sequences:
    if len(sequences) > 1000:
        sequences = sequences[:1000]
    max_1000_art_sequences.append(sequences)

In [34]:
art_len = []
for i in range(1000):
    art_len.append(len(max_1000_art_sequences[i]))
print("max", max(art_len))
print("min", min(art_len))
print("ave", sum(art_len)/len(art_len))

max 1000
min 75
ave 273.675


In [0]:
max_1000_sum_sequences = []
for sequences in sum_sequences:
    if len(sequences) > 1000:
        sequences = sequences[:1000]
    max_1000_sum_sequences.append(sequences)

In [36]:
sum_len = []
for i in range(1000):
    sum_len.append(len(max_1000_sum_sequences[i]))
print("max", max(sum_len))
print("min", min(sum_len))
print("ave", sum(sum_len)/len(sum_len))

max 1000
min 28
ave 124.594


#### 2-2-3. Padding: pad_sequences

In [0]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 1000
padded_art_sequences = pad_sequences(max_1000_art_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [40]:
for i in range(5):
    print(len(padded_art_sequences[i]))

1000
1000
1000
1000
1000


In [43]:
padded_art_sequences[0][:10]

array([1411, 2338,  248,   16, 3994,   22,    5, 6483,  165, 1359],
      dtype=int32)

In [0]:
padded_sum_sequences = pad_sequences(max_1000_sum_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [54]:
art_vocab = []
for seqs in max_1000_art_sequences:
    for seq in seqs:
        if seq not in art_vocab:
            art_vocab.append(seq)
len(art_vocab)

23690

In [55]:
sum_vocab = []
for seqs in max_1000_sum_sequences:
    for seq in seqs:
        if seq not in sum_vocab:
        sum_vocab.append(seq)
len(sum_vocab)

16417

#### 2-2-4. One-Hot Vectrizer: dimention is under vocab_size

In [0]:
from keras.preprocessing.text import one_hot
VOCAB_SIZE = 14999 # paddingするから
one_hot_art_sequences = []
for seqs in max_1000_art_sequences:
    text = " ".join(seqs)
    one_hot_text = one_hot(text, n=VOCAB_SIZE)
    one_hot_art_sequences.append(one_hot_text)

In [57]:
print(one_hot_art_sequences[0][:12])
print(one_hot_art_sequences[1][:12])
print(one_hot_art_sequences[2][:12])

[10891, 7320, 7007, 1008, 5342, 1787, 12034, 11065, 13980, 11281, 14867, 12784]
[11929, 14009, 11929, 1008, 10641, 11421, 6113, 7521, 12034, 5496, 8576, 11688]
[10689, 5411, 10891, 7521, 7461, 13619, 14706, 5896, 14867, 13469, 7521, 1221]


In [58]:
print(len(one_hot_art_sequences[0]))
print(len(one_hot_art_sequences[1]))
print(len(one_hot_art_sequences[2]))

309
248
314


In [0]:
one_hot_sum_sequences = []
for seqs in max_1000_sum_sequences:
  text = " ".join(seqs)
  one_hot_text = one_hot(text, n=VOCAB_SIZE)
  one_hot_sum_sequences.append(one_hot_text)

In [60]:
print(one_hot_sum_sequences[0][:12])
print(one_hot_sum_sequences[1][:12])
print(one_hot_sum_sequences[2][:12])

[4512, 10484, 7521, 10822, 8048, 11626, 8375, 4699, 9453, 9966, 14537, 14349]
[3878, 1008, 13510, 7408, 7221, 10056, 8820, 14829, 3157, 9038, 2291, 3170]
[1477, 11626, 9763, 1380, 9090, 9060, 835, 14461, 9966, 7461, 1477, 11626]


In [0]:
for seqs in one_hot_sum_sequences:
  for seq in seqs:
    if seq == 0:
      print("OMG!")
      break

In [0]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 1000
padded_onehot_art_sequences = pad_sequences(one_hot_art_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [63]:
print(len(padded_onehot_art_sequences[0]))
print(len(padded_onehot_art_sequences[1]))
print(len(padded_onehot_art_sequences[2]))

1000
1000
1000


In [0]:
padded_onehot_sum_sequences = pad_sequences(one_hot_sum_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [65]:
print(len(padded_onehot_sum_sequences[0]))
print(len(padded_onehot_sum_sequences[1]))
print(len(padded_onehot_sum_sequences[2]))

1000
1000
1000


In [0]:
padded_onehot_sum_sequences[1][-5:]

In [0]:
import pandas as pd
padded_onehot_summary_sequences = pd.DataFrame(padded_onehot_sum_sequences) 
padded_onehot_summary_sequences.to_csv("padded_onehot_sum_sequences.csv")
from google.colab import files
files.download('padded_onehot_sum_sequences.csv')

In [0]:
padded_onehot_article_sequences = pd.DataFrame(padded_onehot_art_sequences) 
padded_onehot_article_sequences.to_csv("padded_onehot_art_sequences.csv")
from google.colab import files
files.download('padded_onehot_art_sequences.csv')

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = "https://drive.google.com/open?id=1SiYY-f38JTY_x7xPZM751PoLgIqH--qz"

In [3]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1SiYY-f38JTY_x7xPZM751PoLgIqH--qz


In [0]:
link = "https://drive.google.com/open?id=1zkdrg5pq_s5Skynjk3X0GSkp-B2-rB7T"

In [8]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1zkdrg5pq_s5Skynjk3X0GSkp-B2-rB7T


In [10]:
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('padded_onehot_sum_sequences.csv')
sum_df = pd.read_csv('padded_onehot_sum_sequences.csv')
sum_df.drop("Unnamed: 0", axis=1, inplace=True)
sum_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,4512,10484,7521,10822,8048,11626,8375,4699,9453,9966,...,0,0,0,0,0,0,0,0,0,0
1,3878,1008,13510,7408,7221,10056,8820,14829,3157,9038,...,0,0,0,0,0,0,0,0,0,0
2,1477,11626,9763,1380,9090,9060,835,14461,9966,7461,...,0,0,0,0,0,0,0,0,0,0
3,8095,13136,14278,13695,9862,982,10056,12786,12870,7150,...,0,0,0,0,0,0,0,0,0,0
4,14037,9966,10824,9640,4372,10364,3170,14829,9202,12034,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('padded_onehot_art_sequences.csv')
ar_ticle_df = pd.read_csv('padded_onehot_art_sequences.csv')

In [6]:
ar_ticle_df.drop("Unnamed: 0", axis=1, inplace=True)
ar_ticle_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,10891,7320,7007,1008,5342,1787,12034,11065,13980,11281,...,0,0,0,0,0,0,0,0,0,0
1,11929,14009,11929,1008,10641,11421,6113,7521,12034,5496,...,0,0,0,0,0,0,0,0,0,0
2,10689,5411,10891,7521,7461,13619,14706,5896,14867,13469,...,0,0,0,0,0,0,0,0,0,0
3,9296,3698,12870,14461,8095,13136,14278,13695,9862,982,...,0,0,0,0,0,0,0,0,0,0
4,9853,13136,13847,982,10056,8984,13847,145,1187,4699,...,0,0,0,0,0,0,0,0,0,0


#### 2-2-5. Reshape: manual max_len * one-Hot matrix

In [5]:
import pandas as pd
art_df = pd.read_csv("padded_onehot_art_sequences.csv")
sum_df = pd.read_csv("padded_onehot_sum_sequences.csv")

In [7]:
art_df.drop("Unnamed: 0", axis=1, inplace=True)
sum_df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
sum_list = []
for i in range(len(sum_df)):
    sum_list.append(sum_df.iloc[i].values.tolist())

In [9]:
sum_list[0][:5]

[4512, 10484, 7521, 10822, 8048]

In [10]:
art_list = []
for i in range(len(art_df)):
    art_list.append(art_df.iloc[i].values.tolist())

In [11]:
art_list[1][-5:]

[0, 0, 0, 0, 0]

In [12]:
len(art_list)

2225

In [2]:
import numpy as np
art_matrix = np.zeros((2225, 1000, 15000), dtype='float32')
art_matrix.shape

(2225, 1000, 15000)

In [3]:
sum_matrix = np.zeros((2225, 1000, 15000), dtype='float32')
sum_matrix.shape

(2225, 1000, 15000)

##### 3D input data (samples, maxlen, vocab_size)

In [13]:
for i, seqs in enumerate(art_list):
    for j, seq in enumerate(seqs):
        art_matrix[i, j, seq] = 1.

In [14]:
for i, seqs in enumerate(sum_list):
    for j, seq in enumerate(seqs):
        sum_matrix[i, j, seq] = 1.

##### 2D input data (maxlen, vocab_size)

In [15]:
encoder_inputs = np.zeros((2225, 1000), dtype='float32')
encoder_inputs.shape

(2225, 1000)

In [17]:
decoder_inputs = np.zeros((2225, 1000), dtype='float32')
decoder_inputs.shape

(2225, 1000)

In [16]:
for i, seqs in enumerate(art_list):
    for j, seq in enumerate(seqs):
        encoder_inputs[i, j] = seq

In [18]:
for i, seqs in enumerate(sum_list):
    for j, seq in enumerate(seqs):
        decoder_inputs[i, j] = seq

In [19]:
decoder_inputs.shape

(2225, 1000)

In [20]:
decoder_inputs

array([[ 4512., 10484.,  7521., ...,     0.,     0.,     0.],
       [ 3878.,  1008., 13510., ...,     0.,     0.,     0.],
       [ 1477., 11626.,  9763., ...,     0.,     0.,     0.],
       ...,
       [13064.,   982., 13178., ...,     0.,     0.,     0.],
       [14971., 13573., 11792., ...,     0.,     0.,     0.],
       [10056.,  9966.,  3527., ...,     0.,     0.,     0.]],
      dtype=float32)

##  I found.... 辞書作らないとだめや！
![](https://images.unsplash.com/photo-1486002113024-43b2ce358eb0?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1072&q=80)

### 2-2. B right way of tokenization: Tokenizer
1. Tokenize and One-Hot : Tokenizer
2. Vocabraly: article and summary 15000 words 
3. Padding: pad_sequences 1000 max_len
4. Reshape: manual max_len * one-hot matrix 

In [40]:
bbc_art_sum = pd.read_csv("cleaned_bbc_news.csv")
bbc_art_sum.drop("Unnamed: 0", axis=1, inplace=True)
bbc_art_sum.head()

,row_article,summary
0,continues rapid economy has expanded by a brea...,overall investment in fixed assets was still u...
1,deccan seals deccan has ordered airbus planes ...,government has given its backing to cheaper an...
2,job growth continues in us created fewer jobs ...,creation was one of last main concerns for the...
3,owner buys rival for retail giant federated de...,retail giant federated department stores is to...
4,secures giant japan is to supply japan airline...,chose the after carefully considering both it ...


In [42]:
articles = list(bbc_art_sum.row_article)
summaries = list(bbc_art_sum.summary)

### 2-2-1. Tokenize: text_to_word_sequence

In [50]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 14999
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(articles)
article_sequences = tokenizer.texts_to_sequences(articles)
art_word_index = tokenizer.word_index
len(art_word_index)

23914

In [67]:
print(article_sequences[0][:20])
print(article_sequences[1][:20])
print(article_sequences[2][:20])

[1411, 2338, 248, 16, 3994, 22, 5, 6483, 165, 1359, 50, 966, 4, 120, 967, 176, 118, 505, 38, 2339]
[5211, 8881, 5211, 16, 2233, 3001, 3441, 6, 5, 217, 18, 60, 1270, 7874, 6, 1, 827, 5211, 11, 108]
[478, 196, 1411, 6, 54, 736, 2283, 498, 50, 164, 6, 24, 349, 17, 9, 1, 3322, 6, 5213, 11]


### 2-2-2. Vocabraly: article and summary 15000 words

In [58]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1

In [59]:
counter

14999

In [60]:
tokenizer.fit_on_texts(summaries)
summary_sequences = tokenizer.texts_to_sequences(summaries)
sum_word_index = tokenizer.word_index
len(sum_word_index)

23929

In [61]:
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1

In [62]:
counter

14999

### 2-2-3. Padding: pad_sequences 1000 max_len

In [69]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 1000
pad_art_sequences = pad_sequences(article_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [70]:
print(len(article_sequences[1]), len(pad_art_sequences[1]))

243 1000


In [71]:
pad_sum_sequences = pad_sequences(summary_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [72]:
print(len(summary_sequences[1]), len(pad_sum_sequences[1]))

90 1000


### 2-2-4. Reshape: manual max_len * one-hot matrix

1. Reshape in 3D

In [77]:
import numpy as np
art_matrix = np.zeros((2225, 1000, 15000), dtype='float32')
art_matrix.shape

(2225, 1000, 15000)

In [78]:
for i, seqs in enumerate(pad_art_sequences):
    for j, seq in enumerate(seqs):
        art_matrix[i, j, seq] = 1.

In [79]:
art_matrix[1,2]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [80]:
sum_matrix = np.zeros((2225, 1000, 15000), dtype='float32')

for i, seqs in enumerate(pad_sum_sequences):
    for j, seq in enumerate(seqs):
        sum_matrix[i, j, seq] = 1.

In [82]:
len(sum_matrix[1,2])

15000

In [85]:
len(sum_matrix[1, ])

1000

2. Reshape in 2D

In [90]:
encoder_inputs = np.zeros((2225, 1000), dtype='float32')
encoder_inputs.shape

(2225, 1000)

In [91]:
decoder_inputs = np.zeros((2225, 1000), dtype='float32')
decoder_inputs.shape

(2225, 1000)

In [92]:
decoder_outputs = np.zeros((2225, 1000, 15000), dtype='float32')
decoder_outputs.shape

(2225, 1000, 15000)

In [93]:
for i, seqs in enumerate(pad_art_sequences):
    for j, seq in enumerate(seqs):
        encoder_inputs[i, j] = seq

In [94]:
for i, seqs in enumerate(pad_sum_sequences):
    for j, seq in enumerate(seqs):
        decoder_inputs[i, j] = seq

In [95]:
for i, seqs in enumerate(pad_sum_sequences):
    for j, seq in enumerate(seqs):
        decoder_outputs[i, j, seq] = 1.

In [96]:
decoder_outputs.shape

(2225, 1000, 15000)

3. Reshape in 2D

In [124]:
encoder_inputs2D = np.zeros((1000, 15000), dtype='float32')
encoder_inputs2D.shape

(1000, 15000)

In [125]:
decoder_inputs2D = np.zeros((1000, 15000), dtype='float32')
decoder_inputs2D.shape

(1000, 15000)

In [126]:
for i, seqs in enumerate(pad_art_sequences):
    for j, seq in enumerate(seqs):
        encoder_inputs2D[i, j] = seq

IndexError: index 1000 is out of bounds for axis 0 with size 1000

In [ ]:
for i, seqs in enumerate(pad_sum_sequences):
    for j, seq in enumerate(seqs):
        decoder_inputs2D[i, j] = seq

### 2-2-5. Pre-trained word2vec and word2vec Matrix

In [86]:
embeddings_index = {}
with open('glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [87]:
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [88]:
art_embedding_matrix = embedding_matrix_creater(200, word_index=art_word_index_1500)
art_embedding_matrix.shape

(15000, 200)

In [89]:
sum_embedding_matrix = embedding_matrix_creater(200, word_index=sum_word_index_1500)
sum_embedding_matrix.shape

(15000, 200)

In [99]:
encoder_embedding_layer = Embedding(input_dim = 15000, 
                                    output_dim = 200,
                                    input_length = MAX_LEN,
                                    weights = [art_embedding_matrix],
                                    trainable = False)

In [100]:
decoder_embedding_layer = Embedding(input_dim = 15000, 
                                    output_dim = 200,
                                    input_length = MAX_LEN,
                                    weights = [sum_embedding_matrix],
                                    trainable = False)

In [101]:
sum_embedding_matrix.shape

(15000, 200)

## Step 3. Building Encoder-Decoder Model

In [26]:
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split
import logging

import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import pydot


import keras
from keras import backend as k
k.set_learning_phase(1)
from keras.preprocessing.text import Tokenizer
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

Using TensorFlow backend.


In [103]:
# en_shape = np.shape(train_data["article"][0])
# den_shape = np.shape(train_data["summaries"][0])

ART_MAX_LEN = 1000
SUM_MAX_LEN = 1000
EMBEDDING_DIM = 200
HIDDEN_UNITS = 200

LEARNING_RATE = 0.002
BATCH_SIZE = 32
EPOCHS = 5

# rmsprop = RMSprop(lr=LEARNING_RATE, clipnorm=1.0)

In [118]:
def Recursive_modelA(encoder_embedding_layer, dencoder_embedding_layer):
    """
    Encoder-Decoder-seq2seq
    """
    # encoder
    encoder_inputs = Input(shape=(MAX_LEN, ))
    encoder_embedding = encoder_embedding_layer(encoder_inputs)
    encoder_LSTM = LSTM(200)(encoder_embedding)
    
    # thought_vector = RepeatVector(SUM_MAX_LEN)(encoder_LSTM)
    # rev_encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
    # 
    # encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    # rev_encoder_outputs, rev_state_h, rev_state_c = rev_encoder_LSTM(encoder_embedding)
    #
    # final_state_h = Add()([state_h, rev_state_h])
    # final_state_c = Add()([state_c, rev_state_c])

    # encoder_states = [final_state_h, final_state_c]

    # decoder
    decoder_inputs = Input(shape=(MAX_LEN, ))
    decoder_embedding = decoder_embedding_layer(decoder_inputs)
    decoder_LSTM = LSTM(200)(decoder_embedding)
    
    merge_layer = concatenate([encoder_LSTM, decoder_LSTM])
    # decoder_outputs, _, _, = decoder_LSTM(decoder_embedding, initial_state=encoder_states)
    decoder_outputs = Dense(units=VOCAB_SIZE+1, activation="softmax")(merge_layer) # SUM_VOCAB_SIZE, sum_embedding_matrix.shape[1]
    # decoder_outputs = decoder_dense(decoder_outputs)

    # modeling
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    return model

In [119]:
model = Recursive_modelA(encoder_embedding_layer, decoder_embedding_layer)

In [120]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1000, 200)    3000000     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1000, 200)    3000000     input_6[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LS

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='RecursiveModelA.png')
from google.colab import files
files.download('RecursiveModelA.png')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [121]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

## Step 4. Training your model and Validate it

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(art_matrix, sum_matrix, test_size=0.2)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [115]:
y_train.shape

(1780, 1000)

In [116]:
BATCH_SIZE = 32
EPOCHS = 5

In [123]:
history = model.fit([x_train, y_train], y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=([x_test, y_test], y_test))

ValueError: Error when checking target: expected dense_2 to have shape (15000,) but got array with shape (1000,)

## Step 5. Inference phase and Generate summary from fuyure data